In [26]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
import pandas as pd
from typing import *

from packages.pkl_operations.pkl_io import store_results_dynamic, load_pkl_from_path

In [5]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]"])
tokenizer.pre_tokenizer = Whitespace()

In [11]:
# def iterate_msa_lines(dia_data_path):
#     frame = pd.read_csv(dia_data_path, sep='\t')
#     msa_frame = frame[frame['label']=="MSA"]
#     return iter(msa_frame['content'])

# dia_path = "/project/rrg-mageed/DataBank/DIA_AR_data_bin/ARLU_Binary_train_data_id_light.tsv"
# msa_iter = iterate_msa_lines(dia_path)
# tokenizer.train_from_iterator(msa_iter, trainer)

In [13]:
vocab = set([])
msa_iter = iterate_msa_lines(dia_path) # resetting the iterator
for line in msa_iter:
    tokens = tokenizer.encode(line).tokens
    vocab.update(tokens)
msa_vocab = vocab


In [16]:
store_results_dynamic(msa_vocab, "msa_vocab", "results")

In [21]:
def iterate_lines(dia_data_path, label):
    frame = pd.read_csv(dia_data_path, sep='\t')
    msa_frame = frame[frame['label']==label]
    return iter(msa_frame['content'])

def get_vocab(dia_fname: str, label: str) -> Set[str]:
    """Get the vocabulary for language/dialect.

    Args:
        dia_fname (str): _description_
        label (_type_): _description_
    
    Returns:
        Vocabulary for the language or dialect.
    """
    line_iter = iterate_lines(dia_fname, label)
    trainer = BpeTrainer(special_tokens=["[UNK]"])
    tokenizer.train_from_iterator(line_iter, trainer)
    
    vocab = set([])
    dia_iter = iterate_lines(dia_path, label) # resetting the iterator
    for line in dia_iter:
        tokens = tokenizer.encode(line).tokens
        vocab.update(tokens)
    return vocab

In [24]:
dia_vocab = get_vocab(dia_path, 'DA')
store_results_dynamic(dia_vocab, 'dia_vocab', 'results')

In [28]:
msa_vocab = load_pkl_from_path('results/2022-03-10/msa_vocab')
dia_vocab = load_pkl_from_path('results/2022-03-10/dia_vocab')
len(msa_vocab.intersection(dia_vocab)) / len(msa_vocab.union(dia_vocab))

0.3590010405827263